In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.datasets import load_boston
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.linear_model import SGDRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

In [2]:
X = load_boston().data
Y = load_boston().target

In [5]:
from sklearn.model_selection import train_test_split

In [51]:
X_tr, X_te, y_tr, y_te = train_test_split(X, Y, test_size=.33)

In [52]:
scaler = preprocessing.StandardScaler().fit(X_tr)
X_tr = scaler.transform(X_tr)
X_te = scaler.transform(X_te)

In [256]:
clf = SGDRegressor()
clf.fit(X_tr, y_tr)
print(mean_squared_error(y_te, clf.predict(X_te)))

29.34912618030447


In [258]:
clf.coef_

array([-0.67131374,  0.73570362, -0.38956152,  1.0434513 , -0.59723018,
        3.71428904, -0.36757663, -1.60213802,  0.64994237, -0.61341334,
       -1.83933121,  0.88604001, -3.2187112 ])

In [61]:
import numpy as np
print(np.random.normal(size=13))

[-1.4750848  -0.10730317  0.15914453  0.82333546 -1.49003437  0.76196931
 -0.7618005  -0.81995008 -0.15696079  1.68614223 -1.89538147 -0.958241
 -1.6754991 ]


In [298]:
class CustomSGD:
    def __init__(self, r, iters, k):
        self.w = None
        self.b = None
        self.X = None
        self.y = None
        self.r = r
        self.k = k
        self.iters = iters
        
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.w = np.random.normal(size = X.shape[1])
        self.b = np.random.normal()
        
        self.findBestWeights()
        self.findBestBias()
    
    def findBestWeights(self):
        opt_weights = self.w
        r = self.r
        
        for i in range(self.iters - 1):
            temp = opt_weights
            opt_weights = opt_weights + r * 2 * ( self.selectKPointsforWeights( opt_weights )/self.X.shape[0] )
            
            if np.sum(temp - opt_weights) < 1:
                self.w = temp
                return
            
            r /= 2
        
        self.w = opt_weights
            
    def selectKPointsforWeights(self, w):
        idxs = np.random.randint(0, self.X.shape[0], size=self.k)
        
        k_sum = np.zeros( self.X.shape[1] )
        
        for i in idxs:
            k_sum += self.X[i] * ( self.y[i] - np.dot(w, self.X[i]) - self.b )
            
        return k_sum
        
    def findBestBias(self):
        opt_b = self.b
        r = self.r
        
        for i in range(self.iters - 1):
            temp = opt_b
            opt_b = opt_b + r * 2 * ( self.selectKPointsforBias( self.w )/self.X.shape[0] )
            
            if (temp - opt_b) < 1:
                self.b = temp
                return
                
            r /= 2
        
        self.b = opt_b
            
    def selectKPointsforBias(self, w):
        idxs = np.random.randint(0, self.X.shape[0], size=self.k)
        
        k_sum = 0
        
        for i in idxs:
            k_sum += ( self.y[i] - np.dot(w, self.X[i]) - self.b )
            
        return k_sum
        
    def predict(self, X):
        lst = list()
        for idx in range(X.shape[0]):
            lst.append( np.round(np.dot(self.w, X[idx]) + self.b, 2) )
        return np.array(lst)

In [299]:
model = CustomSGD(10, 10000, 200)

In [308]:
model.fit(X_tr, y_tr)
model.w

array([-45.75371209,  66.63671084, -57.57780168,  19.87842452,
       -61.70987735,  40.83395464, -51.22356611,  48.44280685,
       -63.55567824, -66.24369487, -74.37882169,  15.20967261,
       -78.12643868])

In [309]:
print(mean_squared_error(y_te, model.predict(X_te)))

367218.24307005986


In [297]:
print(y_te[:10])
print( model.predict(X_te)[:10])

[20.6 20.1 14.8 21.2 14.4 50.  20.6 13.  23.3 34.9]
[ 141.58  162.1  -147.53   13.83 -218.73  -84.86   -7.43 -361.7   296.68
  486.18]
